In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
#pos = "rb"

In [3]:
def get_per_game(df):
    cols = df.columns
    
    for c in cols:
        if "RUSHING" in c or "RECEIVING" in c or "FL" in c or "PASSING" in c:
            df[c+"/G"] = df[c]/df["G_MISC"]
            #new_names.append()
    
    return df

In [4]:
def find_last_ewm(x):
    #print(x.columns)
    #print(x.name)
    if x.dtype != "O" and x.name!="pid":
        return x.ewm(halflife=2).mean().values[-1]
        
            
    #elif x.dtype == "O":
     #   values.append(" ")
    #return values
    #print(x.ewm(alpha=0.5))

In [46]:
def find_pct_inc(x, z):
    #print(x.shift(-1))
    pct_inc = []
    for c in x.columns:
        
        if c == "Season":
            pct_inc.append(x[c].values[-1]+1)
        elif x[c].dtype!="O":
            
            pct_inc.append((x[c]/ x[c].shift(1)).values[-1])
        else:
            pct_inc.append(x[c].values[-1])
    return pct_inc

In [101]:
def find_comp_df(pos, season_int, how="past"):
    if how =="ns":
        past_data = pd.read_csv(f"raw data/{pos}_raw.csv")
        ns_data = pd.read_csv(f"ns data/{pos}_ns.csv")
        std_dev = ns_data[["Yahoo", "Sleeper", "RTSports"]].std(axis=1)
        
        past_data = past_data[past_data.Season != past_data.Season.values[-1]]
        ns_data = ns_data.rename({"Rank":"Rank_y"}, axis=1)
        line = pd.concat([past_data, ns_data])#.append(ns_data)
        line = line.drop("Player", axis=1)
        line = line.rename({"Player Team (Bye)": "Player"}, axis=1)
        
        
    else:
        line = pd.read_csv(f"raw data/{pos}_raw.csv")
    #return line
    if pos!= "qb":
        base_line = ["Player", "pid", "Season", "ATT_RUSHING", "YDS_RUSHING", "TD_RUSHING", "REC_RECEIVING", "TGT_RECEIVING", "YDS_RECEIVING", "TD_RECEIVING", "FL_MISC", "AVG", "G_MISC", "FPTS/G_MISC", "FPTS_MISC"]
    else:
        base_line = ["Player", "pid", "Season", 'CMP_PASSING', 'ATT_PASSING', 'PCT_PASSING','YDS_PASSING', 'Y/A_PASSING', 'TD_PASSING', 'INT_PASSING','SACKS_PASSING', 'ATT_RUSHING', 'YDS_RUSHING', 'TD_RUSHING', "FL_MISC", "AVG", "G_MISC", "FPTS/G_MISC", "FPTS_MISC"]

    gs = line[base_line]
    #return gs
    pg_df = get_per_game(gs.copy())
    return pg_df

    if pos != "qb":
        classes = ["_RUSHING", "_RECEIVING"]
        for c in classes:

            if c == "_RUSHING":
                pg_df["YDS/ATT"+c] =pg_df["YDS"+c] /pg_df["ATT"+c]
                pg_df["TD/ATT"+c] =pg_df["TD"+c] /pg_df["ATT"+c]
            else:
                pg_df["YDS/REC"+c] =pg_df["YDS"+c] /pg_df["REC"+c]
                pg_df["YDS/TGT"+c] =pg_df["YDS"+c] /pg_df["TGT"+c]
                pg_df["REC/TGT"+c] =pg_df["REC"+c] /pg_df["TGT"+c]
                pg_df["TD/TGT"+c] =pg_df["TD"+c] /pg_df["TGT"+c]

    agg_feats_base = pg_df.drop("AVG", axis=1).columns[3:].values

    agg_feats = []
    for f in agg_feats_base:
        if "/" in f:
            agg_feats.append(f)
    
    #return agg_feats
    adp = "AVG"
    if how == "ns":
        seasons = np.arange(2018, season_int+1)
    else:
        seasons = np.arange(2018, season_int)

    full_bdf = pd.DataFrame()
    full_bdfs = []
    for s in tqdm(seasons):
        bdf = pg_df[pg_df.Season == s][["pid", "Player", "Season","AVG", "FPTS/G_MISC"]]
        bdf = bdf.rename({"FPTS/G_MISC": "FPTS_TG"}, axis=1)
        #print(len(bdf))
        bdf =  bdf.merge(pg_df[(pg_df.Season < s) & (pg_df.Season > s-4)].groupby("pid")[agg_feats].mean().reset_index(), how="left", on="pid")

        #return pg_df[(pg_df.Season < s) & (pg_df.Season > s-5)]
        ewm_df = pg_df[(pg_df.Season < s) & (pg_df.Season > s-4)].reset_index(drop=True).groupby("pid").agg(find_last_ewm)[agg_feats]

        bdf =  bdf.merge(ewm_df.reset_index(), how="left", on="pid", suffixes=["_avg", "_ewm"])

        #ls = pg_df[(pg_df.Season == s-1)].groupby("pid").mean()[agg_feats]
        #ls_cols = [a+"_ls" for a in ls.columns]        #ls.columns = ls_cols

        #bdf = bdf.merge(ls.reset_index(), how="left", on="pid")


        #full_bdf = full_bdf.append(bdf)
        full_bdfs.append(bdf)

    full_bdf = pd.concat(full_bdfs)
    drafted = full_bdf[~full_bdf.AVG.isna()]

    is_rookie = []
    career_years = []
    
    for i, s in zip(drafted.pid.values, drafted.Season.values):
        last_apps = pg_df[(pg_df.Season < s) & (pg_df.pid == i)]
        if len(last_apps) > 0:
            is_rookie.append(0)
        else:
            is_rookie.append(1)


        career_years.append(len(last_apps))
    
    drafted = drafted.copy()
    drafted.loc[:, "Rookie"] = is_rookie
    drafted.loc[:, "Career_Years"] = career_years
    
    if how == "ns":
        drafted = drafted[drafted.Season == season_int]
        std_dev = std_dev.fillna(std_dev.dropna().tail(5).mean())
        drafted["STD"] = std_dev
    return drafted

In [105]:
pos_data[~pos_data.AVG.isna()]

,Player,pid,Season,ATT_RUSHING,YDS_RUSHING,TD_RUSHING,REC_RECEIVING,TGT_RECEIVING,YDS_RECEIVING,TD_RECEIVING,...,FPTS/G_MISC,FPTS_MISC,ATT_RUSHING/G,YDS_RUSHING/G,TD_RUSHING/G,REC_RECEIVING/G,TGT_RECEIVING/G,YDS_RECEIVING/G,TD_RECEIVING/G,FL_MISC/G
543,Todd Gurley II (FA),13912,2018,256,1251,17,59,81,580,4,...,24.5,342.6,18.285714,89.357143,1.214286,4.214286,5.785714,41.428571,0.285714,0.071429
544,Saquon Barkley (NYG),17240,2018,261,1307,11,91,121,721,4,...,21.3,340.3,16.312500,81.687500,0.687500,5.687500,7.562500,45.062500,0.250000,0.000000
545,Christian McCaffrey (SF),16393,2018,219,1098,7,107,124,867,6,...,20.8,332.0,13.687500,68.625000,0.437500,6.687500,7.750000,54.187500,0.375000,0.062500
546,Alvin Kamara (NO),16421,2018,194,883,14,81,105,709,4,...,20.9,313.7,12.933333,58.866667,0.933333,5.400000,7.000000,47.266667,0.266667,0.000000
547,Ezekiel Elliott (FA),15498,2018,304,1434,6,77,95,567,3,...,19.4,290.6,20.266667,95.600000,0.400000,5.133333,6.333333,37.800000,0.200000,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1793,Zach Evans (LAR),23122,2023,0,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1798,Tank Bigsby (JAC),22908,2023,0,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,Deuce Vaughn (DAL),22984,2023,0,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1802,Jahmyr Gibbs (DET),22968,2023,0,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
pos_data = find_comp_df('rb', 2023)

In [77]:
pos_data.to_csv(f"processed data/rb_proc_data.csv", index=False)

In [78]:
wr_data = find_comp_df('wr', 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  2.91s/it]


In [82]:
wr_data.to_csv(f"processed data/wr_proc_data.csv", index=False)

In [87]:
te_data = find_comp_df('te', 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.75s/it]


In [88]:
te_data.to_csv(f"processed data/te_proc_data.csv", index=False)

In [89]:
qb_data = find_comp_df("qb", 2023)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.00it/s]


In [90]:
qb_data.to_csv(f"processed data/qb_proc_data.csv", index=False)

In [67]:
ns_proc = find_comp_df('rb', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:16<00:00,  2.75s/it]


In [68]:
ns_proc.to_csv("processed data/rb_ns_proc.csv", index=False)

In [83]:
wr_proc = find_comp_df('wr', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:18<00:00,  3.12s/it]


In [86]:
wr_proc.to_csv("processed data/wr_ns_proc.csv", index=False)

In [91]:
te_proc =  find_comp_df('te', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.85s/it]


In [92]:
te_proc.to_csv("processed data/te_ns_proc.csv", index=False)

In [93]:
qb_proc =  find_comp_df('qb', 2023, how="ns")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:06<00:00,  1.02s/it]


In [94]:
qb_proc.to_csv("processed data/qb_ns_proc.csv", index=False)